In [1]:
# DATA LOADING

from utils.data_loader.mobility_data_loader import mobility_data_loader 

# Process whole dataset in to intermediate form
dataframe = mobility_data_loader('20240101','20240101')

1
20240101 dataframe shape
(85653, 18)


In [2]:
df = dataframe['20240101']

In [3]:
df.shape

(85653, 18)

In [4]:
from utils.folium_mapper.ishikawa_region_mapper import ishikawa_region_mapper
map, jpn_adm2 = ishikawa_region_mapper()

In [ ]:
map 

In [5]:
from utils.data_filter.data_travel_filter_by_region import data_travel_filter_by_region

df = data_travel_filter_by_region(df, jpn_adm2)
df.shape

e:\IRP_noto_earthquake\.venv\lib\site-packages\IPython\core\interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
e:\IRP_noto_earthquake\.venv\lib\site-packages\IPython\core\interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


(3807, 18)

In [ ]:
display(df)

In [40]:
# Getting no home data or travels
df_na_home = df[df['longitude_h'].isna() | df['latitude_h'].isna()]
df_na_home.shape

(1107, 18)

In [ ]:
display(df_na_home)

In [ ]:
from utils.plotter.travel_plotter import travel_plotter

# Plot NA home travels
map_travel_of_na_home = travel_plotter(map,df_na_home)
map_travel_of_na_home


In [41]:
# Count the number of unique common_id values in df_na_home
unique_common_ids_na_home = df_na_home['common_id'].nunique()

print("Number of unique common_id in df_na_home:", unique_common_ids_na_home)
# Will check later

Number of unique common_id in df_na_home: 430


In [10]:
# Drop rows where 'longitude_h' or 'latitude_h' is NA
df_with_homes = df.dropna(subset=['longitude_h', 'latitude_h'])
df_with_homes.shape

(2700, 18)

In [11]:
# Filter data with home location inside only
map, jpn_adm2 = ishikawa_region_mapper()

from utils.data_filter.data_home_filter_by_region import data_home_travel_filter_by_region
df_home_inside = data_home_travel_filter_by_region(df_with_homes, jpn_adm2, wantResidents=True)

e:\IRP_noto_earthquake\.venv\lib\site-packages\IPython\core\interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df_home_inside.shape

(1431, 18)

In [ ]:
from utils.plotter.home_plotter import home_plotter
# Observe homes inside the region

home_plot_inside = home_plotter(map, df_home_inside)
home_plot_inside

In [45]:
# Count the number of unique common_id values in df_na_home
unique_common_ids_home_inside = df_home_inside['common_id'].nunique()

print("Number of unique common_id in df_na_home:", unique_common_ids_home_inside)

Number of unique common_id in df_na_home: 514


In [33]:
# Observe evacuations
df_home_inside.shape

(1431, 18)

In [34]:
display(df_home_inside.head())

,common_id,latitude_o,longitude_o,arrive_time_o,stay_time_o,depart_time_o,move_time,mode,latitude_d,longitude_d,arrive_time_d,stay_time_d,depart_time_d,purpose_flg,latitude_h,longitude_h,latitude_w,longitude_w
96,fd308e61cb354debdc1ab4715535c2f1,37.036356,136.977469,2023-12-31 11:31:43,750,2024-01-01 00:02:08,776,car,36.899128,136.770464,2024-01-01 12:58:23,18,2024-01-01 13:16:04,6,37.034292,136.978813,37.034292,136.978813
157,5047cbe833eb8ca3192c2d22a2a56956,37.384122,136.905903,2023-12-31 23:05:41,58,2024-01-01 00:03:40,21,NaN,37.393040,136.902238,2024-01-01 00:25:07,640,2024-01-01 11:04:54,2,37.317625,136.890062,37.384292,136.902563
158,03c6ddacda0b7ede17e770a878cc624b,37.421814,137.248674,2023-12-31 23:40:46,23,2024-01-01 00:03:43,7,walk,37.421814,137.248674,2024-01-01 00:10:46,23,2024-01-01 00:33:43,6,37.420125,137.240063,37.420125,137.240063
325,4b4309d14948a8a43e0039e9dcd85fed,37.384610,136.890164,2023-12-31 03:06:42,1260,2024-01-01 00:07:08,23,car,37.384646,136.890143,2024-01-01 00:30:45,750,2024-01-01 13:00:18,6,37.384375,136.888813,<NA>,<NA>
353,bf680d8d577305205c24278d457fcf23,37.019133,136.961598,2023-12-31 22:53:07,75,2024-01-01 00:07:56,54,walk,37.019133,136.961598,2024-01-01 01:02:02,86,2024-01-01 02:28:09,6,37.018458,136.955062,37.042625,136.966313


In [40]:
map, jpn_adm2 = ishikawa_region_mapper()


In [39]:
import folium
import pandas as pd

def destination_plotter(map_obj, filtered_df):
    map = map_obj  # Assign map_obj to map for clarity

    # Define color categories and corresponding time ranges in minutes
    color_categories = {
        'green': (60, 'Up to 1 hour'),
        'blue': (180, 'Up to 3 hours'),
        'orange': (360, 'Up to 6 hours'),
        'red': (720, 'Up to 12 hours'),
        'purple': (1440, 'Up to 24 hours'),
        'black': (2880, 'More than 24 hours')
    }

    # Create a legend
    legend_html = '''
         <div style="position: fixed; 
                     bottom: 50px; left: 50px; width: 150px; height: 150px; 
                     border:2px solid grey; z-index:9999; font-size:14px;
                     background-color:white;
                     ">&nbsp; <b> Legend </b> <br>'''

    for color, (threshold, description) in color_categories.items():
        legend_html += f'&nbsp; <i class="fa fa-circle fa-1x" style="color:{color}"></i> {description}<br>'
    legend_html += '</div>'
    
    map.get_root().html.add_child(folium.Element(legend_html))

    # Plot circles for each point from filtered_df based on stay_time_d
    for index, row in filtered_df.iterrows():
        stay_time = row['stay_time_d']
        # if stay_time <= 60:
        #     continue  # Skip plotting for stay_time_d <= 60

        # Determine the color category based on stay_time_d
        color = 'black'  # Default color for stay_time_d > 2880
        popup_text = f"Home: {row['common_id']}<br>Stay time: {stay_time} minutes"

        for col, (threshold, description) in color_categories.items():
            if stay_time <= threshold:
                color = col
                popup_text += f"<br>Category: {description}"
                break

        # Plot the circle on the map
        folium.Circle(
            location=[row['latitude_d'], row['longitude_d']],
            radius=20,  # Adjust radius as needed (in meters)
            popup=popup_text,
            color=color,
            fill=True,
            fill_color=color
        ).add_to(map)

    return map

In [41]:
map = destination_plotter(map, df_home_inside)

In [42]:
map

In [25]:
import pandas as pd
# Get the list of residents
# Get unique common_ids
resident_common_ids = df_home_inside['common_id'].unique()



# Create a new DataFrame with only common_id column
resident_list = pd.DataFrame(resident_common_ids, columns=['common_id'])

display(resident_list)

,common_id
0,fd308e61cb354debdc1ab4715535c2f1
1,5047cbe833eb8ca3192c2d22a2a56956
2,03c6ddacda0b7ede17e770a878cc624b
3,4b4309d14948a8a43e0039e9dcd85fed
4,bf680d8d577305205c24278d457fcf23
...,...
509,cdec71406b6229634bd9626754476052
510,c416b04ae8da5e4515e316b1f89b043a
511,9db4ecb0cdc7454d9e729410205405ed
512,3a6cfa376269ed80a41e6f47de4bae6d


In [26]:
# Process whole dataset in to intermediate form
dataframe = mobility_data_loader('20240102','20240102')

1
2
20240102 dataframe shape
(84857, 18)


In [27]:
df = dataframe['20240102']

In [28]:
from utils.folium_mapper.ishikawa_region_mapper import ishikawa_region_mapper
map, jpn_adm2 = ishikawa_region_mapper()

In [30]:
df = data_travel_filter_by_region(df, jpn_adm2)
df.shape

e:\IRP_noto_earthquake\.venv\lib\site-packages\IPython\core\interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
e:\IRP_noto_earthquake\.venv\lib\site-packages\IPython\core\interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


(3481, 18)

In [31]:
# Drop rows where 'longitude_h' or 'latitude_h' is NA
df_with_homes = df.dropna(subset=['longitude_h', 'latitude_h'])
df_with_homes.shape

(2425, 18)

In [32]:
# Filter data with home location inside only
map, jpn_adm2 = ishikawa_region_mapper()

from utils.data_filter.data_home_filter_by_region import data_home_travel_filter_by_region
df_home_inside = data_home_travel_filter_by_region(df_with_homes, jpn_adm2, wantResidents=True)

e:\IRP_noto_earthquake\.venv\lib\site-packages\IPython\core\interactiveshell.py:3577: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
import pandas as pd
# Get the list of residents
# Get unique common_ids
resident_common_ids = df_home_inside['common_id'].unique()



# Create a new DataFrame with only common_id column
resident_list2 = pd.DataFrame(resident_common_ids, columns=['common_id'])

display(resident_list2)

,common_id
0,973095433aa47270ee2a50fcfd949334
1,7980b0db88943b81fd6e006440ea8286
2,f164e703b15efe7bee9b8b4cfe5494b4
3,1c5a544eae382d9a63b74a3ded32cdbf
4,ef877c048be314bc7c00aea021231b03
...,...
451,4549506a0d2300fb4ad0a354455524a5
452,bb3aff6ba84309044e7b23cd2d1af5a8
453,344c984e7691f292710088c96c0ca3dd
454,66d1b63bb7d03abaffb3c3a1856a3f4c


In [34]:
common_ids_count = len(set(resident_list['common_id']).intersection(set(resident_list2['common_id'])))
print(common_ids_count)

355
